<a href="https://colab.research.google.com/github/Hiromitsu4676/Pytorch/blob/main/face_leaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

     |████████████████████████████████| 308 kB 5.4 MB/s 
     |████████████████████████████████| 80 kB 5.6 MB/s 
     |████████████████████████████████| 210 kB 31.6 MB/s 
     |████████████████████████████████| 75 kB 2.0 MB/s 
     |████████████████████████████████| 149 kB 39.4 MB/s 
     |████████████████████████████████| 113 kB 29.9 MB/s 
     |████████████████████████████████| 49 kB 2.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=a69d4bc6d1434feffb2bbf3157e9199e5d6380b593e2169572b08f807d020433
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import optuna
from torchvision import transforms
from PIL import Image
import xml.etree.ElementTree as ET 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
class Xml2List(object):
    
    def __init__(self, classes):
        self.classes = classes
        
    def __call__(self, xml_path):
        
        ret = []
        
        xml = ET.parse(xml_path).getroot()
        
        for size in xml.iter("size"):
          
            width = float(size.find("width").text)
            height = float(size.find("height").text)
                
        for obj in xml.iter("object"):
            
            difficult = int(obj.find("difficult").text)
            if difficult == 1:
                continue
                
            bndbox = [width, height]
            
            name = obj.find("name").text.lower().strip() 
            bbox = obj.find("bndbox") 
            
            pts = ["xmin", "ymin", "xmax", "ymax"]
            
            for pt in pts:
                
                cur_pixel =  float(bbox.find(pt).text)
                    
                bndbox.append(cur_pixel)
                
            label_idx = self.classes.index(name)
            bndbox.append(label_idx)
            
            ret += [bndbox]
            
        return np.array(ret) # [width, height, xmin, ymin, xamx, ymax, label_idx]

In [5]:
class MyDataset(torch.utils.data.Dataset):
    
    def __init__(self, df, image_dir):
        
        super().__init__()
        
        self.image_ids = df["image_id"].unique()
        self.df = df
        self.image_dir = image_dir
        
    def __getitem__(self, index):
 
        transform = transforms.Compose([
                                        transforms.ToTensor()
        ])
 
        # 入力画像の読み込み
        image_id = self.image_ids[index]
        image = Image.open(f"{self.image_dir}/{image_id}.jpg")
        image = transform(image)
        
        # アノテーションデータの読み込み
        records = self.df[self.df["image_id"] == image_id]
        boxes = torch.tensor(records[["xmin", "ymin", "xmax", "ymax"]].values.astype(np.float32), dtype=torch.float32)
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        labels = torch.tensor(records["class"].values.astype(np.int64), dtype=torch.int64)
        
        iscrowd = torch.zeros((records.shape[0], ), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"]= labels
        target["image_id"] = torch.tensor([index])
        target["area"] = area
        target["iscrowd"] = iscrowd


        return image, labels[0], image_id
    
    def __len__(self):
        return self.image_ids.shape[0]

In [6]:
# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(800*600*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 5),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y, z) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 損失誤差を計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 5 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y ,z in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [8]:
def objective(trial):
    #学習率
    lr = trial.suggest_uniform('lr', 1e-4, 1e-1)

    #エポック数
    epochs = trial.suggest_discrete_uniform('epochs', 3, 5, 1)

    optimizer = torch.optim.SGD(model.parameters(),lr)
    
    print('学習率',lr)
    print('エポック数',epochs)

    for step in range(int(epochs)):
        print('step:',step,'epoch:',epochs)
        train(train_dataloader, model, loss_fn, optimizer)
        test_loss = test(test_dataloader, model)

    return test_loss

In [9]:
xml_paths = glob.glob("./drive/MyDrive/Pytorch/privateDataset/xml_resize/*.xml")
classes = ["saito", "ohnaka","doi",'sugai','suzuki']
    
transform_anno = Xml2List(classes)

df = pd.DataFrame(columns=["image_id", "width", "height", "xmin", "ymin", "xmax", "ymax", "class"])

In [10]:
for path in xml_paths:
    image_id = path.split("/")[-1].split(".")[0]
    bboxs = transform_anno(path)
    
    for bbox in bboxs:
        tmp = pd.Series(bbox, index=["width", "height", "xmin", "ymin", "xmax", "ymax", "class"])
        tmp["image_id"] = image_id
        df = df.append(tmp, ignore_index=True)

df = df.sort_values(by="image_id", ascending=True)

df.head()

,image_id,width,height,xmin,ymin,xmax,ymax,class
8,1E5D4546-308B-4BC9-BDFE-954ADB3BD279,800.0,600.0,62.0,0.0,288.0,253.0,1.0
14,4808BBD0-26F8-4519-B2A0-DD26884984D2,800.0,600.0,453.0,85.0,640.0,343.0,3.0
13,4808BBD0-26F8-4519-B2A0-DD26884984D2,800.0,600.0,162.0,162.0,314.0,391.0,1.0
48,65299796_43d6f8c57280e7a0cd014e931b99ed86_2110...,800.0,600.0,295.0,141.0,488.0,368.0,0.0
49,65299796_c23f0ac9a76b18d1570dc47c995cd6ce_2110...,800.0,600.0,290.0,204.0,473.0,393.0,0.0


In [11]:
image_dir = "./drive/MyDrive/Pytorch/privateDataset/img_resize"
dataset = MyDataset(df, image_dir)

In [12]:
batch_size = 1
train_dataloader = DataLoader(dataset, batch_size=batch_size)
test_dataloader = DataLoader(dataset, batch_size=batch_size)

In [13]:
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [14]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1440000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=5, bias=True)
    (5): ReLU()
  )
)


In [15]:
loss_fn = nn.CrossEntropyLoss()

TRIAL_SIZE = 3
study = optuna.create_study()
study.optimize(objective, n_trials=TRIAL_SIZE)

#最適化したハイパーパラメータの結果
study.best_params

[I 2022-02-09 23:31:40,400] A new study created in memory with name: no-name-cbca8eae-f274-424d-b821-27628524098b


学習率 0.08513079108678435
エポック数 3.0
step: 0 epoch: 3.0
loss: 1.638454  [    0/   35]
loss: 0.000000  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

step: 1 epoch: 3.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

step: 2 epoch: 3.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]


[I 2022-02-09 23:32:13,804] Trial 0 finished with value: 1.6094379425048828 and parameters: {'lr': 0.08513079108678435, 'epochs': 3.0}. Best is trial 0 with value: 1.6094379425048828.


Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

学習率 0.07921208780592964
エポック数 5.0
step: 0 epoch: 5.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

step: 1 epoch: 5.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

step: 2 epoch: 5.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

step: 3 epoch: 5.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5

[I 2022-02-09 23:32:57,880] Trial 1 finished with value: 1.6094379425048828 and parameters: {'lr': 0.07921208780592964, 'epochs': 5.0}. Best is trial 0 with value: 1.6094379425048828.


Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

学習率 0.06831306232688568
エポック数 4.0
step: 0 epoch: 4.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

step: 1 epoch: 4.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

step: 2 epoch: 4.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5/   35]
loss: 1.609438  [   10/   35]
loss: 1.609438  [   15/   35]
loss: 1.609438  [   20/   35]
loss: 1.609438  [   25/   35]
loss: 1.609438  [   30/   35]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 

step: 3 epoch: 4.0
loss: 1.609438  [    0/   35]
loss: 1.609438  [    5

[I 2022-02-09 23:33:33,080] Trial 2 finished with value: 1.6094379425048828 and parameters: {'lr': 0.06831306232688568, 'epochs': 4.0}. Best is trial 0 with value: 1.6094379425048828.


Test Error: 
 Accuracy: 40.0%, Avg loss: 1.609438 



{'epochs': 3.0, 'lr': 0.08513079108678435}

In [16]:
#最適化後の目的関数の値
study.best_value

1.6094379425048828

In [17]:
#全試行過程
study.trials

[FrozenTrial(number=0, values=[1.6094379425048828], datetime_start=datetime.datetime(2022, 2, 9, 23, 31, 40, 407213), datetime_complete=datetime.datetime(2022, 2, 9, 23, 32, 13, 804468), params={'lr': 0.08513079108678435, 'epochs': 3.0}, distributions={'lr': UniformDistribution(high=0.1, low=0.0001), 'epochs': DiscreteUniformDistribution(high=5.0, low=3.0, q=1.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None),
 FrozenTrial(number=1, values=[1.6094379425048828], datetime_start=datetime.datetime(2022, 2, 9, 23, 32, 13, 809611), datetime_complete=datetime.datetime(2022, 2, 9, 23, 32, 57, 880656), params={'lr': 0.07921208780592964, 'epochs': 5.0}, distributions={'lr': UniformDistribution(high=0.1, low=0.0001), 'epochs': DiscreteUniformDistribution(high=5.0, low=3.0, q=1.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None),
 FrozenTrial(number=2, values=[1.609437942504882